In [44]:
import pandas as pd
import geopandas as gpd
import json
import IPython.display

In [45]:
rows_per_geotype = {}

with open('vaarweginformatie/vaarweginformatie.jsonl') as f:
    for line in f:
        obj = json.loads(line)
        geo_type = obj['GeoType']
        rows = rows_per_geotype.get(geo_type, [])
        rows.append(obj)
        rows_per_geotype[geo_type] = rows        
        

In [56]:
dfs_per_geotype = {}
gdfs_per_geotype = {}
for geo_type, rows in rows_per_geotype.items():
    df = pd.DataFrame(rows)
    dfs_per_geotype[geo_type] = df
    if 'Geometry' in df.columns:
        geometry = gpd.GeoSeries.from_wkt(df['Geometry'].astype(str), on_invalid='warn')
        gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')
        gdfs_per_geotype[geo_type] = gdf
        missing_geometry = gdf[pd.isna(gdf['geometry'])]
        if missing_geometry.shape[0]:
            IPython.display.display(missing_geometry)
        

/Users/baart_f/.virtualenvs/py312/lib/python3.12/site-packages/shapely/io.py:282: Warning: Invalid WKT: geometry is returned as None. ParseException: Expected word but encountered number: 'nan'
  return lib.from_wkt(geometry, invalid_handler, **kwargs)


,Id,GeoType,GeoGeneration,Name,Geometry,VinCode,VhfChannel,HeadOfficeId,Note,ForeignCode,geometry
106,16498083,vtssector,511,Wybelsum Radar,NaN,51431,21,10848.0,NaN,NaN,None
107,16498207,vtssector,511,Knock Radar,NaN,51429,20,10848.0,NaN,NaN,None
108,16498535,vtssector,511,Sector Brienenoord,NaN,50851,21,19000.0,NaN,NaN,None
109,16501410,vtssector,511,Verkeerscentrale Scheveningen,NaN,51370,21,21079.0,NaN,NaN,None
110,16497765,vtssector,2027,Borkum Radar,NaN,NaN,18,10848.0,NaN,65102221,None


/Users/baart_f/.virtualenvs/py312/lib/python3.12/site-packages/shapely/io.py:282: Warning: Invalid WKT: geometry is returned as None. ParseException: Expected word but encountered number: 'nan'
  return lib.from_wkt(geometry, invalid_handler, **kwargs)


,Id,GeoType,GeoGeneration,Name,VinCode,Direction,Geometry,RouteKmEnd,EndJunctionId,RouteKmBegin,StartJunctionId,Length,TypeId,FairwayId,RouteId,ForeignCode,geometry
1416,1429068,section,18,Vaarwegvak van 0 tot 0,65100003,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,1429065,1429062,NaN,None
1449,1429204,section,18,Vaarwegvak van 0 tot 0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,1429193,1429189,DE65100003,None
1450,1429575,section,18,Vaarwegvak van 0 tot 0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,1429572,1429560,AT65100003,None
1451,1430221,section,18,Vaarwegvak van 0 tot 0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,1430174,1429872,BE65100003,None


/Users/baart_f/.virtualenvs/py312/lib/python3.12/site-packages/shapely/io.py:282: Warning: Invalid WKT: geometry is returned as None. ParseException: Expected word but encountered number: 'nan'
  return lib.from_wkt(geometry, invalid_handler, **kwargs)


,Id,GeoType,GeoGeneration,Name,VinCode,AdministrationId,WorkingAreaId,VhfChannel,City,TelephoneNumber,EmailAddress,Note,Geometry,ForeignCode,geometry
0,55262,headoffice,322,Verkeerscentrale Stad,50852,19454,14169.0,,Rotterdam,010-2522701,,NaN,NaN,NaN,None
1,39509,headoffice,322,Verkeerspost Hartel,54397,19454,14169.0,,Rotterdam,010-2522613,,Verkeerspost Hartel valt onder de Verkeerscent...,NaN,NaN,None


In [57]:
for name, df in dfs_per_geotype.items():
    df.to_json(f'data/{name}.json')
    df.to_parquet(f'data/{name}.parquet')
for name, gdf in gdfs_per_geotype.items():
    gdf.to_file(f'data/{name}.geojson')
    gdf.to_parquet(f'/data/{name}.geoparquet')
    